# Import necessary libraries

In [39]:
import gym
import numpy as np
import time
from IPython import display
import time

In [4]:
env = gym.make('FrozenLake-v1')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [5]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [6]:
env.observation_space.n

16

In [7]:
env.action_space.n

4

In [8]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [9]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 55)

In [ ]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

error: ignored

In [10]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [11]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 44/1000
Average number of steps: 12.477272727272727


In [13]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 93/1000
Average number of steps: 16.397849462365592


In [14]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 742/1000
Average number of steps: 36.23450134770889


# Policy Evaluation Function

In [15]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [16]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [17]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [18]:
np.all(v_values_1 >= v_values_0)

True

In [ ]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [ ]:
np.all(v_values_2 >= v_values_1)

True

In [ ]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [ ]:
np.all(v_values_3 >= v_values_2)

True

# Value Iteration Function

In [19]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [22]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [23]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

# Policy Extraction Function

In [20]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

In [24]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

<ipython-input-20-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


In [25]:
optimal_policy
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 746/1000
Average number of steps: 36.91018766756032


In [26]:
play_multiple_times(env, policy_1, 1000)

Number of successes: 49/1000
Average number of steps: 11.510204081632653


# Policy Iteration Function

In [48]:
def policy_iteration(env,max_iters=500, gamma=0.9):
  # initialize
  policy = np.zeros(env.observation_space.n)
  for i in range(max_iters):
    # Compute new value of policy evaluation 
    new_value = policy_evaluation(env,policy)
    # Extracted new policy from the computed value
    new_policy = policy_extraction(env,new_value)
    # check convergence
    if np.all(np.isclose(policy,new_policy)):
      print(f'Policy-Iteration converged at {i}-th iteration.')
      break
    policy = new_policy
  return policy

In [21]:
optimal_policy_1 = policy_iteration(env)
play_multiple_times(env,optimal_policy_1,1000)

Converged at 0-th iteration.
Converged at 23-th iteration.
Converged at 59-th iteration.
Converged at 62-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Policy-Iteration converged at 5-th iteration.


<ipython-input-20-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


Number of successes: 754/1000
Average number of steps: 37.3448275862069


# Compare between Value Iteraion and Policy Iteration

Value Iteration

In [47]:
print('FROZEN-LAKE-V1 USING VALUE ITERATION')
s = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)
play_multiple_times(env, optimal_policy, 1000)
e = time.time()
print('Executable time:',e-s)

FROZEN-LAKE-V1 USING VALUE ITERATION
Converged at 79-th iteration.


<ipython-input-20-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


Number of successes: 741/1000
Average number of steps: 38.232118758434545
Executable time: 0.8577351570129395


In [41]:
print('FROZEN-LAKE-8X8-V0 USING VALUE ITERATION')
env1 = gym.make('FrozenLake8x8-v1')
s = time.time()
optimal_v_values = value_iteration(env1, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env1, optimal_v_values, gamma=0.9)
play_multiple_times(env1, optimal_policy, 1000)
e = time.time()
print('Executable time:',e-s)

FROZEN-LAKE-8X8-V0 USING VALUE ITERATION


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Converged at 117-th iteration.


<ipython-input-20-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


Number of successes: 727/1000
Average number of steps: 70.47317744154057
Executable time: 1.70347261428833


In [42]:
print('TAXI-V3 USING VALUE ITERATION')
env2= gym.make('Taxi-v3')
s = time.time()
optimal_v_values = value_iteration(env2, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env2, optimal_v_values, gamma=0.9)
play_multiple_times(env2, optimal_policy, 1000)
e = time.time()
print('Executable time:',e-s)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


TAXI-V3 USING VALUE ITERATION
Converged at 116-th iteration.


<ipython-input-20-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


Number of successes: 1000/1000
Average number of steps: 13.123
Executable time: 4.39072847366333


Policy Iteration

In [50]:
s = time.time()
optimal_policy_1 = policy_iteration(env)
play_multiple_times(env,optimal_policy_1,1000)
e = time.time()
print('Executable time:',e-s)

Converged at 0-th iteration.
Converged at 23-th iteration.
Converged at 59-th iteration.
Converged at 62-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Policy-Iteration converged at 5-th iteration.


<ipython-input-20-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


Number of successes: 728/1000
Average number of steps: 37.58379120879121
Executable time: 0.8271799087524414


In [51]:
s = time.time()
optimal_policy_1 = policy_iteration(env1)
play_multiple_times(env1,optimal_policy_1,1000)
e = time.time()
print('Executable time:',e-s)

Converged at 27-th iteration.
Converged at 91-th iteration.
Converged at 92-th iteration.
Converged at 86-th iteration.
Converged at 90-th iteration.
Converged at 92-th iteration.
Converged at 95-th iteration.


<ipython-input-20-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


Converged at 100-th iteration.
Converged at 112-th iteration.
Converged at 117-th iteration.
Policy-Iteration converged at 9-th iteration.
Number of successes: 753/1000
Average number of steps: 72.18857901726427
Executable time: 2.159482955932617


In [45]:
s = time.time()
optimal_policy_1 = policy_iteration(env2)
play_multiple_times(env2,optimal_policy_1,1000)
e = time.time()
print('Executable time:',e-s)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Converged at 88-th iteration.


<ipython-input-20-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


Converged at 97-th iteration.
Converged at 100-th iteration.
Converged at 101-th iteration.
Converged at 102-th iteration.
Converged at 103-th iteration.
Converged at 106-th iteration.
Converged at 109-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Policy-Iteration converged at 16-th iteration.
Number of successes: 1000/1000
Average number of steps: 13.164
Executable time: 7.745406150817871


# Observation of two algorithms when running on three gym game environments

Kết quả chạy:

- FrozenLake-V1: 

  VI: Converged at 79th iteration, NoS: 741/1000, Step: 38.232118758434545, Time :  0.8577351570129395

  PI: Converged at 5th iteration, NoS: 728/1000, Step: 37.58379120879121, Time: 0.8271799087524414


- FrozenLake8x8-V1:

  VI: Converged at 117th iteration, NoS: 727/1000, Step: 70.47317744154057, Time :  1.70347261428833

  PI: Converged at 9th iteration, NoS: 753/1000, Step: 72.18857901726427, Time: 2.159482955932617

- Taxi-V3:

  VI: Converged at 116th iteration, NoS: 1000/1000, Step: 13.123, Time : 4.39072847366333

  PI: Converged at 16th iteration, NoS: 1000/1000, Step:13.164, Time: 7.745406150817871


Nhận xét:
- Policy Iteration hội tụ khá sớm so với Value Iteration (Policy Iteration sẽ đi tính nhiều giá trị hơn và cho ra nhiều policy hơn nên sẽ nhanh chóng hội tụ so với Value Iteration chỉ đi 1 lần đến khi nào hội tụ)

-  Vì Policy Iteration đi tìm policy tốt nhất (tìm đến khi nào hội tụ) nên số vòng lặp trong Policy Iteration sẽ nhiều hơn so với Value Iteration. 

-  Policy Iteration và Value Iteration cho ra số lần thành công, bước đi trung bình và thời gian gần giống nhau cụ thể là ở FrozenLake-V1 thì số lần thành công và bước đi trung bình không chênh nhau quá nhiều. Còn ở hai game còn lại là FrozenLake8x8-V1 và Taxi-V3 không xuất hiện sự khác biệt nhiều. 

- Ở FrozenLake8x8-V1 thì số lần thành công của PI lớn hơn VI 26 lần, còn thời gian thì chậm hơn khoảng 0.4s. 

- Cuối cùng là Taxi-V3, với thời gian chạy của Policy Iteration lớn hơn gần gấp đôi so với thời gian chạy của Value Iteration (Vì Policy Iteration xài khá nhiều vòng lặp)

- Policy Iteration là một thuật toán chạy đến khi nào hội tụ và chiến lược bước đi được cập nhật và quá trình này lặp lại liên tục. Ngược lại, với Value Iteration thì chỉ chạy mỗi bước đi, tìm ra giá trị của chiến lược đó và nó sẽ thực hiện hành động có giá trị lớn nhất. Nên Value Iteration sẽ nặng về mặt tính toán và lưu trữ vì phải lưu lại các giá trị tương ứng với mỗi bước. 

- Tóm lại, với việc sử dụng môi trường gym game, ta có thể thấy Value Iteration nhanh hơn Policy Iteration về mặt thời gian, còn kết quả thì khá giống nhau. Trên thực tế thì Policy Iteration sẽ nhanh hơn Value Iteration (có thể là về mặt hội tụ) tùy thuộc vào môi trường mà thuật toán được sử dụng. Nên không chắc về việc kết luận thuật toán nào sẽ tốt hơn cái còn lại cả. 